Кольцов Кирилл, группа 209, детектор смены сцен на основе метрики DSSIM и метода опорных векторов. 

Были проведены множественные эксперименты, основанные на метриках, полученных из предыдущего этапа задания. Наилучший результат получился при использовании в качестве тренировочного датасета для метода опорных векторов значений, полученных при продсчете метрики DSSIM для двух соседних изображений, разбитых на 16 блоков. Также проводились тесты (1), в которых: 
- использовалась DSSIM, но при разбиении на 4, 100 блоков
- использовалась метрика, основанная на сравнении гистограмм при разбиении на 4, 16, 100 блоков
- использовались средние значения двух описанных выше метрик, вместе и по отдельности, посчитанные по значениям, полученным для 4, 16, 100 блоков.

Помимо варьирования обучающих данных, в процессе тестов (1) сравнивались различные методы машинного обучения. Лучший результат показал метод опорных векторов (scikit learn: SVC) с параметрами {'C': 5, 'gamma': 'scale', 'kernel': 'linear'}. Также хорошие показатели выдавала логистическая регрессия (LogisticRegression). Хуже работали градиентный бустинг (GradientBoostingClassifier), метод случайного леса (RandomForestClassifier), метод ближайших соседей (NearestNeighbors). Также данные стандартизировались с помощью StandardScaler и MinMaxScaler, но это не принесло результатов

In [ ]:
# GRADED CELL: scene_change_detector_ml

def scene_change_detector_ml(frames, with_vis = False):
    vis = []
    metric_values = []
    
    ###
    def view_as_blocks(frame, number_of_blocks):
        H, W, C = frame.shape
        block_h = H // number_of_blocks   
        block_w = W // number_of_blocks 

        # Обрезка кадра до размеров, кратных блокам
        H_cropped = block_h * number_of_blocks
        W_cropped = block_w * number_of_blocks
        frame = frame[:H_cropped, :W_cropped, :]

        # Разбиение на блоки и перестановка осей
        blocks = frame.reshape(
            number_of_blocks, block_h,
            number_of_blocks, block_w,
            C
        ).transpose(0, 2, 1, 3, 4)

        return blocks

    # Вычисление ssim для блоков 
    def compute_ssim_for_blocks(prev_block, block, k1=0.01, k2=0.03, L=255):
        block = cv2.cvtColor(block, cv2.COLOR_BGR2GRAY).astype(np.float64)
        prev_block = cv2.cvtColor(prev_block, cv2.COLOR_BGR2GRAY).astype(np.float64)

        C1 = (k1 * L) ** 2
        C2 = (k2 * L) ** 2

        mu1 = np.mean(block, axis=(0, 1))
        mu2 = np.mean(prev_block, axis=(0, 1))
        mu1_sq = mu1 ** 2
        mu2_sq = mu2 ** 2
        mu1_mu2 = mu1 * mu2
        
        sigma1_sq = np.var(block, axis=(0, 1))
        sigma2_sq = np.var(prev_block, axis=(0, 1)) 
        sigma12 = np.mean(block * prev_block, axis=(0, 1)) - mu1_mu2
        
        numerator = (2 * mu1_mu2 + C1) * (2 * sigma12 + C2)
        denominator = (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
        
        if denominator == 0:
            ssim = 1.0 if numerator == 0 else 0.0
        else:
            ssim = numerator / denominator

        return ssim


    def calculate_dssim_vector(prev_frame, frame, number_of_blocks=10):
        frame = view_as_blocks(frame, number_of_blocks)
        prev_frame = view_as_blocks(prev_frame, number_of_blocks)

        dssim_vector = []
        
        for i in range(0, number_of_blocks):
            for j in range(0, number_of_blocks):
                block = frame[i, j]
                prev_block = prev_frame[i, j]
                
                block_ssim = compute_ssim_for_blocks(prev_block, block)
                dssim_vector.append((1 - block_ssim) / 2)
        
        return dssim_vector


    def euristic_scene_change_detector(frames):
        metric_values = []
        prev_frame = None
        number_of_blocks = 4
        
        for idx, frame in tqdm(enumerate(frames), leave=False):
            if idx == 0:
                prev_frame = frame
                metric_values.append([0 for _ in range(number_of_blocks**2)])
                continue

            dssim_current = calculate_dssim_vector(prev_frame, frame, number_of_blocks=number_of_blocks)
            metric_values.append(dssim_current)
                
            prev_frame = frame

        return metric_values
    ###
    
    
    ### START CODE HERE ###
    X_test = euristic_scene_change_detector(frames) 
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        ### START CODE HERE ###
        ###  END CODE HERE  ###
        pass

    model = pickle.load(open("model.pkl", 'rb')) 
    predict_cuts = model.predict(X_test)
    
    return np.where(predict_cuts == 1)[0], vis, metric_values